In [8]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

In [10]:
# --- 1. Daten laden ---
url = "https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv"
df = pd.read_csv(url, parse_dates=["Date"])
df = df.rename(columns={
    "AAPL.Open":   "Open",
    "AAPL.High":   "High",
    "AAPL.Low":    "Low",
    "AAPL.Close":  "Close",
    "AAPL.Volume": "Volume"
})
df["Ticker"] = "AAPL"
df = df[["Date", "Ticker", "Open", "High", "Low", "Close", "Volume"]]
df.sort_values(["Ticker", "Date"], inplace=True)

tickers = df["Ticker"].unique()

# --- 2. Dash App initialisieren ---
app = dash.Dash(__name__)
app.title = "Apple Trading Dashboard (px)"

# --- 3. Layout ---
app.layout = html.Div([
    html.H1("Interaktives Trading Dashboard – Apple (AAPL)"),

    html.Div([
        html.Label("Aktie auswählen:"),
        dcc.Dropdown(
            id="ticker-dropdown",
            options=[{"label": t, "value": t} for t in tickers],
            value=tickers[0],
            clearable=False
        )
    ], style={"width": "30%", "display": "inline-block"}),

    html.Div([
        html.Label("Datumsbereich wählen:"),
        dcc.DatePickerRange(
            id="date-picker",
            start_date=df["Date"].min(),
            end_date=df["Date"].max(),
            display_format="YYYY-MM-DD"
        )
    ], style={"width": "60%", "display": "inline-block", "paddingLeft": "20px"}),

    dcc.Graph(id="price-chart"),
    dcc.Graph(id="stats-chart"),
])

# --- 4. Callback mit Plotly Express ---
@app.callback(
    [Output(component_id="price-chart", component_property="figure"),
     Output(component_id="stats-chart", component_property="figure")],
    [Input(component_id="ticker-dropdown", component_property="value"),
     Input(component_id="date-picker", component_property="start_date"),
     Input(component_id="date-picker", component_property="end_date")]
)
def update_graphs(selected_ticker, start_date, end_date):
    # 1) Strings in Timestamps umwandeln
    start = pd.to_datetime(start_date)
    end   = pd.to_datetime(end_date)

    # 2) Filtern nach Ticker + Datum
    mask = (
        (df["Ticker"] == selected_ticker) &
        (df["Date"]    >= start) &
        (df["Date"]    <= end)
    )
    dff = df.loc[mask].copy()

    # 3) Gleitende Kennzahlen berechnen
    dff["MA20"] = dff["Close"].rolling(20).mean()
    dff["Vol20"] = dff["Close"].rolling(20).std()

    # 4) Für px: Reset Index, damit 'Date' Spalte bleibt
    plot_df = dff.reset_index()

    # --- Preis-Chart mit px.line ---
    price_fig = px.line(
        plot_df,
        x="Date",
        y=["Close", "MA20"],
        labels={"value": "Preis (USD)", "variable": "Linie"},
        title=f"{selected_ticker} – Schlusskurs & 20-Tage MA"
    )
    price_fig.update_layout(legend_title_text="")

    # --- Volatilitäts-Chart mit px.bar ---
    stats_fig = px.bar(
        plot_df,
        x="Date",
        y="Vol20",
        labels={"Vol20": "20-Tage Volatilität"},
        title=f"{selected_ticker} – 20-Tage Volatilität"
    )

    return price_fig, stats_fig

# --- 5. Server starten ---
if __name__ == "__main__":
    app.run(debug=True)